### <font color=blue> 加载和创建数据集</font>

In [1]:
import numpy as np
print("加载数据------\n")
train_feature = np.load("data/train_11.npy")
train_label = np.load("data/train_label_11.npy")
test = np.load("data/test_11.npy")
print(train_feature.shape)
print(train_label.shape)
print(test.shape)

加载数据------

(1229932, 429)
(1229932,)
(451552, 429)


In [2]:
import torch
from torch.utils.data import Dataset
class TIMITDataset(Dataset):  # 创建数据集都需要继承Dataset类
    def __init__(self,x,y=None):
        self.data = torch.from_numpy(x).float()
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    
    def __gettiem__(self,idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)

### <font color=blue> 划分训练集和验证集</font>

In [3]:
val_ratio = 0.2  # 设定验证集占数据集的比例
percent = int(train_feature.shape[0] * (1-val_ratio))  # 除去验证集后，训练集还有多少条数据
train_x, train_y, val_x, val_y = train_feature[:percent], train_label[:percent], train_feature[percent:], train_label[percent:]  # 划分训练集和验证集
print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)

(983945, 429) (983945,)
(245987, 429) (245987,)


### <font color=blue> 从dataset中创建一个DataLoader, batch_size可以随意设置</font>

In [4]:
batch_size = 64
from torch.utils.data import DataLoader
train_set = TIMITDataset(train_x,train_y)
val_set = TIMITDataset(val_x,val_y)
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True)  # 只对训练集进行shuffle
val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=False)

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

将不需要的变量从内存中删除

In [ ]:
import gc
del train_feature,train_label,train_x, train_y, val_x, val_y
gc.collect()

### <font color=blue> 创建模型</font>
定义模型结构

In [ ]:
import torch
import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.layer1 = nn.Linear(429,1024)
        self.layer2 = nn.Linear(1024,512)
        self.layer3 = nn.Linear(512,128)
        self.out = nn.Linear(128,39)
        self.act_fn = nn.Sigmoid()
    
    def forward(self,x):
        x = self.layer1(x)
        x = self.act_fn(x)
        x = self.layer2(x)
        x = self.act_fn(x)
        x = self.layer3(x)
        x = self.act_fn(x)
        x = self.out(x)
        return x

### <font color=blue> 开始训练</font>

In [ ]:
# 检测GPU是否可以用
def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 固定随机数的种子
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
same_seeds(0)
device = get_device()
print(device)
num_epoch = 20
learning_rate = 0.0001
model_path = "/.model.ckpt"
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs,labels)
        _, train_pred = torch.max(outputs,1)
        batch_loss.backward()
        optimizer.step()
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # 模型进入验证状态
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

### <font color=blue> 测试</font>

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))